In [1]:
import numpy as np
import pandas as pd
from os import listdir
from os.path import isfile, join, isdir
from tqdm import tqdm

In [2]:
# Load data
data_a = pd.read_parquet("stock_a.parquet")
data_b = pd.read_parquet("stock_b.parquet")

# Extract shared 10 minute time frames
a_times = set(data_a.time_id.unique())
b_times = set(data_b.time_id.unique())

shared_times = a_times.intersection(b_times)

data_a = data_a[data_a["time_id"].isin(shared_times)]
data_b = data_b[data_b["time_id"].isin(shared_times)]

In [3]:
# Reindex 
data_a = data_a.set_index(["time_id", "seconds_in_bucket"])
data_b = data_b.set_index(["time_id", "seconds_in_bucket"])

In [4]:
# Find and forward fill missing timestamps within each 10 minute interval 

missing_a = [[], []]
missing_b = [[], []]

# Find missing (time, second) tuples
for t in tqdm(shared_times): 
    for i in range(600):
        if (t, i) not in data_a.index: 
            missing_a[0].append(t)
            missing_a[1].append(i)

        if (t, i) not in data_b.index: 
            missing_b[0].append(t)
            missing_b[1].append(i)

assert len(missing_a[0]) == len(shared_times) * 600 - len(data_a)
assert len(missing_b[0]) == len(shared_times) * 600 - len(data_b)

# Initialize missing rows to null 
null_rows_a = []
for _ in missing_a[0]: 
    null_rows_a.append([np.nan] * 8)

null_rows_b = []
for _ in missing_b[0]: 
    null_rows_b.append([np.nan] * 8)

# Create indices for missing rows 
index_a = pd.MultiIndex.from_arrays(missing_a, names=["time_id", "seconds_in_bucket"])
index_b = pd.MultiIndex.from_arrays(missing_b, names=["time_id", "seconds_in_bucket"])

# Concatenate data with missing rows 
data_a = pd.concat([data_a, pd.DataFrame(null_rows_a, columns=data_a.columns, index=index_a)])
data_b = pd.concat([data_b, pd.DataFrame(null_rows_b, columns=data_b.columns, index=index_b)]) 

assert len(data_a) == len(shared_times) * 600
assert len(data_b) == len(shared_times) * 600

# Join and forward fill data
data = data_a.join(data_b, lsuffix="_a", rsuffix="_b")
data = data.ffill()
data = data.sort_index(level=["time_id", "seconds_in_bucket"])

100%|██████████| 3830/3830 [00:21<00:00, 176.32it/s]
/var/folders/p0/nb5t36rs6p5_2b9yzyw6y8hw0000gn/T/ipykernel_54429/3093890056.py:34: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  data_a = pd.concat([data_a, pd.DataFrame(null_rows_a, columns=data_a.columns, index=index_a)])
/var/folders/p0/nb5t36rs6p5_2b9yzyw6y8hw0000gn/T/ipykernel_54429/3093890056.py:35: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  data_b = pd.concat([data_b, pd.DataFrame(null_rows_b, columns=data_b.columns, index=index_b)])

In [5]:
data

bid_price1_a  ask_price1_a  bid_price2_a  \
time_id seconds_in_bucket                                             
5       0                      1.001422      1.002301      1.001370   
        1                      1.001422      1.002301      1.001370   
        2                      0.998275      0.998754      0.998179   
        3                      0.998275      0.998754      0.998179   
        4                      0.998275      0.998754      0.998179   
...                                 ...           ...           ...   
32767   595                    0.998275      0.998754      0.998179   
        596                    0.998275      0.998754      0.998179   
        597                    0.998275      0.998754      0.998179   
        598                    0.998275      0.998754      0.998179   
        599                    0.998275      0.998754      0.998179   

                           ask_price2_a  bid_size1_a  ask_size1_a  \
time_id seconds_in_bucket                                           
5       0                      1.002353          3.0        226.0   
        1                      1.002353          3.0        100.0   
        2                      0.998946         92.0         90.0   
        3                      0.998946         92.0         90.0   
        4                      0.998946         92.0         90.0   
...                                 ...          ...          ...   
32767   595                    0.998946         92.0         90.0   
        596                    0.998946         92.0         90.0   
        597                    0.998946         92.0         90.0   
        598                    0.998946         92.0         90.0   
        599                    0.998946         92.0         90.0   

                           bid_size2_a  ask_size2_a  bid_price1_b  \
time_id seconds_in_bucket                                           
5       0                          2.0        100.0      1.000754   
        1                          2.0        100.0      1.000754   
        2                         26.0         28.0      1.000754   
        3                         26.0         28.0      1.000754   
        4                         26.0         28.0      1.000754   
...                                ...          ...           ...   
32767   595                       26.0         28.0      0.998911   
        596                       26.0         28.0      0.998911   
        597                       26.0         28.0      0.998911   
        598                       26.0         28.0      0.998911   
        599                       26.0         28.0      0.998911   

                           ask_price1_b  bid_price2_b  ask_price2_b  \
time_id seconds_in_bucket                                             
5       0                      1.001542      1.000689      1.001607   
        1                      1.001673      1.000689      1.001739   
        2                      1.001411      1.000623      1.001476   
        3                      1.001542      1.000689      1.001607   
        4                      1.001476      1.000623      1.001542   
...                                 ...           ...           ...   
32767   595                    0.999109      0.998812      0.999208   
        596                    0.999109      0.998812      0.999208   
        597                    0.999109      0.998812      0.999208   
        598                    0.999109      0.998812      0.999208   
        599                    0.999109      0.998812      0.999208   

                           bid_size1_b  ask_size1_b  bid_size2_b  ask_size2_b  
time_id seconds_in_bucket                                                      
5       0                          1.0         25.0         25.0        100.0  
        1                         26.0         60.0         25.0        100.0  
        2                          1.0         25.0         25.

In [6]:
# Derive target column 

# Mid prices of a and b
data["a_mid"] = (data["ask_price1_a"] + data["bid_price1_a"]) / 2
data["b_mid"] = (data["ask_price1_b"] + data["bid_price1_b"]) / 2

# Price after 120 seconds
future_vals_a = []
future_vals_b = []
for tid in tqdm(shared_times): 
    for i in range(600): 
        if i > 479: 
            future_vals_a.append(np.nan)
            future_vals_b.append(np.nan)
        else: 
            future_row = data.loc[(tid, i + 120)]
            future_vals_a.append((future_row.bid_price1_a + future_row.ask_price1_a) / 2)
            future_vals_b.append((future_row.bid_price1_b + future_row.ask_price1_b) / 2)

data["mid_a_2min"] = pd.Series(future_vals_a, index=data.index)
data["mid_b_2min"] = pd.Series(future_vals_b, index=data.index)

# a, b percentage return in 120 seconds
data["a_perf"] = data["mid_a_2min"] / data["a_mid"] - 1
data["b_perf"] = data["mid_b_2min"] / data["b_mid"] - 1

# Target column (1 if a outperforms b, 0 otherwise)
data["target"] = data["a_perf"] > data["b_perf"]

100%|██████████| 3830/3830 [01:01<00:00, 62.54it/s]


In [7]:
data = data.astype(np.float64)

# Extract price data 
price_cols = ["bid_price1_a", "ask_price1_a", "bid_price2_a", 
              "ask_price2_a", "bid_size1_a", "ask_size1_a", 
              "bid_size2_a", "ask_size2_a", "bid_price1_b", 
              "ask_price1_b", "bid_price2_b", "ask_price2_b", 
              "bid_size1_b", "ask_size1_b", "bid_size2_b", 
              "ask_size2_b"]
price_data = data[price_cols]

In [8]:
price_data

bid_price1_a  ask_price1_a  bid_price2_a  \
time_id seconds_in_bucket                                             
5       0                      1.001422      1.002301      1.001370   
        1                      1.001422      1.002301      1.001370   
        2                      0.998275      0.998754      0.998179   
        3                      0.998275      0.998754      0.998179   
        4                      0.998275      0.998754      0.998179   
...                                 ...           ...           ...   
32767   595                    0.998275      0.998754      0.998179   
        596                    0.998275      0.998754      0.998179   
        597                    0.998275      0.998754      0.998179   
        598                    0.998275      0.998754      0.998179   
        599                    0.998275      0.998754      0.998179   

                           ask_price2_a  bid_size1_a  ask_size1_a  \
time_id seconds_in_bucket                                           
5       0                      1.002353          3.0        226.0   
        1                      1.002353          3.0        100.0   
        2                      0.998946         92.0         90.0   
        3                      0.998946         92.0         90.0   
        4                      0.998946         92.0         90.0   
...                                 ...          ...          ...   
32767   595                    0.998946         92.0         90.0   
        596                    0.998946         92.0         90.0   
        597                    0.998946         92.0         90.0   
        598                    0.998946         92.0         90.0   
        599                    0.998946         92.0         90.0   

                           bid_size2_a  ask_size2_a  bid_price1_b  \
time_id seconds_in_bucket                                           
5       0                          2.0        100.0      1.000754   
        1                          2.0        100.0      1.000754   
        2                         26.0         28.0      1.000754   
        3                         26.0         28.0      1.000754   
        4                         26.0         28.0      1.000754   
...                                ...          ...           ...   
32767   595                       26.0         28.0      0.998911   
        596                       26.0         28.0      0.998911   
        597                       26.0         28.0      0.998911   
        598                       26.0         28.0      0.998911   
        599                       26.0         28.0      0.998911   

                           ask_price1_b  bid_price2_b  ask_price2_b  \
time_id seconds_in_bucket                                             
5       0                      1.001542      1.000689      1.001607   
        1                      1.001673      1.000689      1.001739   
        2                      1.001411      1.000623      1.001476   
        3                      1.001542      1.000689      1.001607   
        4                      1.001476      1.000623      1.001542   
...                                 ...           ...           ...   
32767   595                    0.999109      0.998812      0.999208   
        596                    0.999109      0.998812      0.999208   
        597                    0.999109      0.998812      0.999208   
        598                    0.999109      0.998812      0.999208   
        599                    0.999109      0.998812      0.999208   

                           bid_size1_b  ask_size1_b  bid_size2_b  ask_size2_b  
time_id seconds_in_bucket                                                      
5       0                          1.0         25.0         25.0        100.0  
        1                         26.0         60.0         25.0        100.0  
        2                          1.0         25.0         25.

In [9]:
# Flatten each 2 minute interval and include target
flattened_data = []
flattened_idx = [[], []]

for tid in tqdm(shared_times): 
    for sib in range(0, 600, 120): 
        chunk = price_data.loc[(tid,sib):(tid,sib+119)]
        chunk = chunk.values.flatten()
        chunk = np.append(chunk, data.loc[(tid,sib+119)]["target"])
        flattened_data.append(chunk)
        flattened_idx[0].append(tid)
        flattened_idx[1].append(sib+119)

flattened_pd_idx = pd.MultiIndex.from_arrays(flattened_idx, names=["time_id", "seconds_in_bucket"])
final_data = pd.DataFrame(flattened_data, index=flattened_pd_idx)

100%|██████████| 3830/3830 [00:01<00:00, 1949.28it/s]


In [10]:
final_data = final_data.sort_index()

In [11]:
# Add column names 
col_names = []
for i in range(120): 
    for stem in price_cols: 
        col_names.append(stem + f"_t={i}")
col_names.append("target")
final_data.columns = col_names

In [12]:
final_data

bid_price1_a_t=0  ask_price1_a_t=0  \
time_id seconds_in_bucket                                       
5       119                        1.001422          1.002301   
        239                        0.998275          0.998754   
        359                        0.998275          0.998754   
        479                        0.998275          0.998754   
        599                        0.998275          0.998754   
...                                     ...               ...   
32767   119                        0.999808          1.000335   
        239                        0.998275          0.998754   
        359                        0.998275          0.998754   
        479                        0.998275          0.998754   
        599                        0.998275          0.998754   

                           bid_price2_a_t=0  ask_price2_a_t=0  \
time_id seconds_in_bucket                                       
5       119                        1.001370          1.002353   
        239                        0.998179          0.998946   
        359                        0.998179          0.998946   
        479                        0.998179          0.998946   
        599                        0.998179          0.998946   
...                                     ...               ...   
32767   119                        0.999760          1.000383   
        239                        0.998179          0.998946   
        359                        0.998179          0.998946   
        479                        0.998179          0.998946   
        599                        0.998179          0.998946   

                           bid_size1_a_t=0  ask_size1_a_t=0  bid_size2_a_t=0  \
time_id seconds_in_bucket                                                      
5       119                            3.0            226.0              2.0   
        239                           92.0             90.0             26.0   
        359                           92.0             90.0             26.0   
        479                           92.0             90.0             26.0   
        599                           92.0             90.0             26.0   
...                                    ...              ...              ...   
32767   119                           90.0            118.0            200.0   
        239                           92.0             90.0             26.0   
        359                           92.0             90.0             26.0   
        479                           92.0             90.0             26.0   
        599                           92.0             90.0             26.0   

                           ask_size2_a_t=0  bid_price1_b_t=0  \
time_id seconds_in_bucket                                      
5       119                          100.0          1.000754   
        239                           28.0          1.002329   
        359                           28.0          1.005413   
        479                           28.0          1.003510   
        599                           28.0          1.004035   
...                                    ...               ...   
32767   119                          100.0          0.999901   
        239                           28.0          0.999604   
        359                           28.0          1.000099   
        479                           28.0          1.000594   
        599                           28.0          1.000198   

                           ask_price1_b_t=0  ...  ask_size2_a_t=119  \
time_id seconds_in_bucket                    ...                      
5       119                        1.001542  ...               20.0   
        239                        1.003248  ...              194.0   
        359                        1.006463  ...               28.0   
        479                        1.003969  ...                2.0   
        599                        

In [13]:
# Split 90% to train, 10% to test
time_ids = list(shared_times)
train_count = len(time_ids) * 9 // 10

train_data = final_data.loc[time_ids[:train_count]]
test_data = final_data.loc[time_ids[train_count:]]

In [14]:
# Assign training data to 5 folds for CV 
train_data["fold"] = -1

In [15]:
# Randomly assign each train_time_id to a fold
train_time_id = train_data.index.get_level_values("time_id").unique().to_numpy()
np.random.seed(0)
np.random.shuffle(train_time_id)
for i, tid in enumerate(train_time_id): 
    train_data.loc[tid, "fold"] = i % 5

In [16]:
train_data

bid_price1_a_t=0  ask_price1_a_t=0  \
time_id seconds_in_bucket                                       
8192    119                        0.999422          0.999711   
        239                        0.999519          0.999952   
        359                        0.998275          0.998754   
        479                        0.998275          0.998754   
        599                        0.998275          0.998754   
...                                     ...               ...   
15486   119                        0.999576          0.999953   
        239                        0.998680          0.999104   
        359                        0.998275          0.998754   
        479                        0.998963          0.999246   
        599                        0.999057          0.999576   

                           bid_price2_a_t=0  ask_price2_a_t=0  \
time_id seconds_in_bucket                                       
8192    119                        0.999374          0.999759   
        239                        0.999422          1.000000   
        359                        0.998179          0.998946   
        479                        0.998179          0.998946   
        599                        0.998179          0.998946   
...                                     ...               ...   
15486   119                        0.999481          1.000142   
        239                        0.998586          0.999151   
        359                        0.998179          0.998946   
        479                        0.998916          0.999293   
        599                        0.999010          0.999670   

                           bid_size1_a_t=0  ask_size1_a_t=0  bid_size2_a_t=0  \
time_id seconds_in_bucket                                                      
8192    119                           90.0            290.0             23.0   
        239                          200.0            130.0            205.0   
        359                           92.0             90.0             26.0   
        479                           92.0             90.0             26.0   
        599                           92.0             90.0             26.0   
...                                    ...              ...              ...   
15486   119                            2.0            190.0            100.0   
        239                           90.0             98.0             27.0   
        359                           92.0             90.0             26.0   
        479                          194.0              5.0            100.0   
        599                          137.0             90.0            100.0   

                           ask_size2_a_t=0  bid_price1_b_t=0  \
time_id seconds_in_bucket                                      
8192    119                          204.0          1.000048   
        239                          100.0          1.000239   
        359                           28.0          0.998709   
        479                           28.0          0.998040   
        599                           28.0          0.998805   
...                                    ...               ...   
15486   119                            4.0          1.000668   
        239                           34.0          1.000286   
        359                           28.0          0.998950   
        479                            2.0          0.998854   
        599                          104.0          0.999045   

                           ask_price1_b_t=0  ...  bid_price1_b_t=119  \
time_id seconds_in_bucket                    ...                       
8192    119                        1.000430  ...            1.000239   
        239                        1.000430  ...            0.998709   
        359                        0.998996  ...            0.998040   
        479                        0.998231  ...            0.998805   
        599                  

In [17]:
# Save data
train_data.to_csv("train.csv", index=False)
test_data.to_csv("test.csv", index=False)